# Lec 04 深度学习基础及扫盲

## 主要内容：
- <a href='#rnn'>1. 循环神经网络RNN</a>
    - <a href='#rnn1'>1.1 场景与多种引用</a>
    - <a href='#dnn_rnn'>1.2 神经网络到循环神经网络RNN</a>
    - <a href='#rnn2'>1.3 NLP文字序列最爱的RNN</a>
        - 双向RNN
        - 深度RNN
    - <a href='#rnn3'>1.4 BPTT算法</a>
    - <a href='#pic'>1.5 看图说话</a>

- <a href='#lstm'>2. LSTM</a>
    - <a href='#lstm1'>2.1 长时依赖问题</a>
    - <a href='#lstm2'>2.2 “记忆细胞”与状态</a>

- <a href='#application'>3.NLP的应用</a>
    - <a href='#application1'>3.1 各式各样的生成模型</a>
    - <a href='#application2'>3.2 看图说话基础版与高级版</a>
    - <a href='#application3'>3.3 序列到序列学习（机器翻译等）</a>

<h2><a name='rnn'>1. 循环神经网络RNN</a></h2>


<h3><a name='rnn1'>1.1 场景与多种引用</a></h3>

在自然语言处理中，另外一个重要的应用领域，就是**文本的自动撰写**。

关键词、关键短语、自动摘要提取都属于这个领域中的一种应用。
不过这些应用，都是**由多到少的生成**。

这里我们介绍其另外一种应用：**由少到多的生成**，包括句子的复写，由关键词、主题生成文章或者段落等。


- 由多到少的生成：
    - 关键词
    - 关键短语
    - 自动摘要

- 又少到多的生成：
    - 句子的复写
    - 由关键词、主题生成文章或者段落

如MarkDown，Latex等这种有模式pattern的语言，都可以通过RNN（循环神经网络）的生成模型，学习到如何生成相应的符合其pattern的产物，如：Markdown格式的笔记，Latex格式的数学公式等。

下面介绍循环神经网络的应用：
- 模仿论文（连公式的格式都是正确的）
- 模仿Linux内核代码“写程序”
- 模仿小四的作品
- 机器翻译（Seq2Seq）
- 看图说话

<img src='./images/rnn1.png' width='30%'/>

这篇生成的论文，在样式上是符合人类的论文的书写习惯，可能内容上没有什么实际的研究意义。

<img src='./images/rnn2.png' width='30%'/>

不同的语言在做编码的时候，语法是不一样的，如java和c需要标记变量的类型，而python则是东岱语言，不需要预先定义变量的类型，而是根据赋值的类型而变化。

学习到编码方式和风格，RNN将序列的模式学会，前后组合在一起，有具体的实际意义。然后RNN可以通过这样的序列，生成相应的序列。如：论文和代码都不一定有实际意义，但是至少都是符合相关的语法和模式的。

<img src='./images/rnn3.png' width='30%'/>

如果序列对应的是中文，那么生成的就是中文。

模型的参数如何；  
给定的数据如何（丰富度，准确性等）。

<img src='./images/rnn4.png' width='30%'/>

序列到序列，Seq2Seq的应用。

对于计算机而言，喂给的是向量，生成的向量，只是向量相应的映射的值。

<img src='./images/rnn5.png' width='30%'/>

这个任务，在后续会讲解。 

中间的一副图：问和答，是更高级的一种。

<h3><a name='dnn_rnn'>1.2 神经网络到循环神经网络RNN</a></h3>

<img src='./images/nn1.png' width='70%'/>


在神经网络中，我们有很多种的结构和名称。

如：
- MLP/DNN： 一般的神经网络结构比较简单，input layer, n * hidden layer , output layer
- CNN卷积神经网络，主要用于处理图像，也可以应用于自然语言处理等多方面
- RNN( LSTM, GRU)： 自然语言处理很多的处理序列

#### 循环神经网络RNN

为什么有BP神经网络，CNN，还要RNN?
- 传统神经网络（包括CNN），输入和输出都是相互独立的。
    - 图像上的猫和狗是分隔开的，但有些任务，后续的输出和之前的内容是相关的。
    - 如：“我是中国人，我的母语是__”
- RNN引入“记忆”的概念
    - 循环2字 来源于其每个元素都执行相同的任务。
    - 但是输出依赖于 “输入”和“记忆”。

传统的MLP/DNN，都是输入和输出相互独立的。

但在，文本的序列问题，前后是有关联的。 这种依赖于上下文，在RNN中引入了“记忆”的概念。

如上面的应用中的看图提问并回答，那么提问的就要存在“记忆”中，然后基于问题去回答。

“循环”相当于一个“回路”，就是一遍一遍地去做。


<img src='./images/rnn6.png' width='70%'/>

- $x_t$是时间t处的输入
- $S_t$是时间t处的“记忆”，$S_t = f(UX_t +WS_{t-1})$，f可以是tanh等
- $O_t$是时间t处的输出，比如：是预测下个词的话，可能是softmax输出的属于每个候选词的概率，$O_t = softmax(VS_t)$

<img src='./images/rnn_e1.png' width='50%'/>

**循环神经网络之 结构细节 **
- 可以把隐状态$S_t$视作“记忆体”，捕捉了之前时间点上的信息。
- 输出$O_t$由当前时间及之前所有的“记忆“共同计算得到。
- 很可惜，实际应用中，$S_t$并不能捕捉和保留之前所有信息（记忆有限？）
    - LSTM可以解决这个问题，长短时记忆网络
- 不同于CNN，这里的RNN其实整个神经网络都共享一组参数（U,V,W），极大减小了需要训练和预估的参数量。
- 图中的$O_t$在有些任务下是不存在的，比如：文本情感分析，其实只需要最后的output结果就行。

<h3><a name='rnn2'>1.3 NLP文字序列最爱的RNN</a></h3>

#### RNN与生成模型

<img src='./images/rnn7.png' width='70%'/>

#### RNN生成模型模仿语言风格例子：

<img src='./images/rnn8.png' width='30%'/>

<img src='./images/rnn_hand.png' width='40%'/>

在所有的RNN或LSTM的模型中，一个点不代表一个neuron，而是代表相对应的维度大小的所有的值。

如：
上图中的$s_{t}$就代表了100x100维的矩阵。

#### 不同类型的RNN
- 双向RNN
    - 有些情况下，当前的输出不知依赖于之前的序列元素，还可能依赖之后的序列元素
    - 比如： 从一段话中踢掉部分词，让你补全。如：完形填空。
    - 直观理解：双向RNN叠加
    <img src='./images/birnn1.png' width='70%'/>


双向RNN（BiRNN)：  
- $\overrightarrow h_t$一个向量捕获了从前往后的信息，$h_t$受时刻t-1的$h_{t-1}$的影响；
- $\overleftarrow h_t$一个向量捕获了从后往前的信息，$h_t$受时刻t+1的$h_{t+1}$的影响；


$U[\overrightarrow h_t;\overleftarrow h_t]$ 将这两个维度一样的向量做一个拼接，拼成一个矩阵U，再拿到最后的结果$y_t$.



在经典的循环神经网络中，状态的传输是从前往后单向的。  
然而，在有些问题中，当前时刻的输出不仅和之前的状态有关系，也和之后的状态相关。  
这时就需要双向RNN（BiRNN）来解决这类问题。

例如：预测一个语句中缺失的单词不仅需要根据前文来判断，也需要根据后面的内容，这时双向RNN就可以发挥它的作用。

双向RNN是由两个RNN上下叠加在一起组成的。输出由这两个RNN的状态共同决定。

从上图可以看出，双向RNN的主题结构就是两个单向RNN的结合。在每一个时刻t，输入会同时提供给这两个方向相反的RNN，而输出则是由这两个单向RNN共同决定（可以拼接或者求和等）。

同样地，将双向RNN中的RNN替换成LSTM或者GRU结构，则组成了BiLSTM和BiGRU。

- 深层双向RNN
    - 和双向RNN的区别是每一步/每个时间点我们设定多层结构
    <img src='./images/birnn2.png' width='70%'/>

在同一个时间点上，反复学习了好几遍之后，得到的$y_t$。

上图就是在同一时间点，学习了3遍之后得到的输出，会反复的消化知识。

深层双向RNN中的W，V可能是共享的，也可能在不同层次之间不是共享的。

<h3><a name='rnn3'>1.4 BPTT算法</a></h3>

- MLP(DNN)与CNN用BP算法求偏导
- BPTT和BP是一个思路，只不过既然有step，就和时间t有关系

<img src='./images/bptt1.png' width='70%'/>

<img src='./images/bptt2.png' width='70%'/>

假定上面的$x_0$表示“我”，$x_1$表示“爱”，$x_2$表示“北京”，$x_3$表示“天安门”，$x_4$表示“广场”。

希望预测的的output是$y_0$为“爱”的对应的向量值最大，同理，$y_1$为“北京”，$y_2$为“天安门”，$y_3$为“广场”，$y_4$为“。”

假设这里的输入$x_i$是一个1x4w大小的one-hot向量，那么对应的输出的$y_i$也是1x4w的softmax的概率向量。其中$\sum y_i=1$。

想要衡量实际的$y_i$的输出和$\hat y_i$的softmax的概率向量的差异，度量模型输出的结果和实际的结果之间的差异的**损失函数**，此处用的是**交叉熵**。

$$E_t(y_t,\hat y_t) = -y_t \log \hat y_t$$

序列的预测，每个位置的损失都需要纳入考量。  
所以，每个位置上的损失最后需要相加。
$$E(y,\hat y)=\sum_{t} E_t(y_t,\hat y_t)$$
$$=-\sum_t y_t \log \hat y_t$$
最终这个才是预测值和标准答案之间的差距。

<img src='./images/bptt3.png' width='70%'/>

沿着时间轴，继续往前（begin)求偏导数。

### <a name='pic'>1.5 看图说话</a>
这个是基于相关的Paper的实际的项目“看图说话”的简单介绍

#### RNN与图片描述输出：

<img src='./images/rnnandpic.png' width='70%'/>

看图说话，我们可能希望能够生成一句话“人带草帽”。

这个网络结构是一个AlexNet的CNN，由卷积层+池化层+卷积层+池化层+卷积层+池化层+卷积层+池化层+2个全连接层FC构成。

全连接层的时候，把最上面输入的图表示成 4096x1的一个向量。

这个向量表示的是图像的内容。

上图任务中，针对图像的内容，去表述生成一句话。如果没有图像4096x1的向量的信息的介入，可以随机生成话语。  

把图像的向量灌进来这个RNN结构中，V只在Start的时候灌进来，后面就不会再添加进来。
在论文里面，这样的效果是最好的。

<img src='./images/rnnandpic2.png' width='70%'/>

#### 图片描述数据集


<img src='./images/coco.png' width='80%'/>

<img src='./images/coco2.png' width='80%'/>

-----

## <a name='lstm'>2. LSTM</a>


前面提到的RNN的解决了，对之前的信息保存的问题。

但是！存在长期依赖的问题。
- 看电影的时候，某些情节的推断需要依赖很久以前的一些细节。
- 很多其他的任务也一样。
- 很可惜随着时间间隔不断增大时，RNN会丧失学习到连接如此远的信息的能力。
- 也就是说，记忆容量有限，一本书从头到尾一字不漏的去记，肯定离得越远的东西忘得越多。
- 怎么办：LSTM

LSTM是RNN的一种，大体结构几乎一样。 

区别是：
- 它的“记忆细胞”改造过。
- 该记的信息会一直传递，不该记的会被“门”截断。

之前提到的RNN结构如下：

<img src='./images/rnn_p.png' width='80%'/>

咱们把“记忆细胞”表示得酷炫有点：
<img src='./images/lstm_ku.png' width='80%'/>

LSTM呢？
- “记忆细胞”变得稍微复杂了一点点
<img src='./images/lstm_memory.png' width='80%'/>

图太复杂，细节看不懂？别着急，我们解释解释。

<img src='./images/lstm_detail.png' width='80%'/>

LSTM关键：“记忆细胞”
- 细胞状态类似于传送带。直接在整个链上面运行，只有一些少量的线性交互。 信息在上面流传保持不变会很容易。

<img src='./images/lstm_cell.png' width='60%'/>

### <a name='lstm1'>2.1 长时依赖问题</a>
### <a name='lstm2'>2.2 “记忆细胞”与状态</a>